<a href="https://colab.research.google.com/github/MarcelloCeresini/QuestionAnswering/blob/dpr/dense_passage_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
username = 'MarcelloCeresini'
repository = 'QuestionAnswering'

# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !pip3 install transformers
    !git clone https://www.github.com/{username}/{repository}.git
    #from google.colab import drive
    #drive.mount('/content/drive/')
    %cd /content/QuestionAnswering/src
    using_TPU = True    # If we are running this notebook on Colab, use a TPU
except:
    IN_COLAB = False
    using_TPU = False   # If you're not on Colab you probably won't have access to a TPU

# Description

In this notebook, we will try to implement the architecture detailed in [Dense Passage Retrieval for Open-Domain Question Answering](https://arxiv.org/pdf/2004.04906.pdf). 

The idea is that we have a corpus of documents $C = {p_1, p_2, \dots, p_M}$ where each passage $p_i$ can be viewed as a sequence of tokens $w_1^{(i)}, w_2^{(i)}, \dots, w_{|p_i|}^{(i)}$ and given a question $q$ we want to find the sequence of tokens $w_s^{(i)}, w_{s+1}^{(i)}, \dots, w_{e}^{(i)}$ from one of the passage $i$ that can answer the question.

In order to find the passage $i$ we need an efficient **Retriever** (i.e. a function $R: (q, C) \rightarrow C_F$ where $C_F$ is a very small set of $k$ documents that have a high correlation with the query.)

In the Tf-Idf example, the retriever was simply a function that returned the top 5 scores obtained by computing the vector cosine similarity between the query and all other documents. The problem with this approach is that it is not very efficient. Tf-Idf is a **sparse** document/query representation, thus computing a multitude of dot products between these very long vectors can be expensive.

The paper cited above proposes a **dense** representation instead. It uses a Dense Encoder $E_P$ which maps all paragraphs to $d$-dimensional vectors. These vectors are stored in a database so that they can be efficiently retrieved. 

At run-time, another Dense Encoder is used $E_Q$ which maps the input question to a vector with the same dimensionality $d$. Then, a similarity score is computed between the two representations:

$sim(p,q) = E_Q(q)^\intercal E_P(p)$

In the paper, $E_Q$ and $E_P$ are two independent BERT transformers and the $d$-dimensional vector is the **output at the $\texttt{[CLS]}$ token** (so, $d = 768$).
- This leaves open the possibility to use a larger dimensionality (eg. concatenating the output at multiple blocks like we did for the QA task).

The $d$-dimensional representations of the $M$ passages are indexed using [FAISS](https://github.com/facebookresearch/faiss), an efficient, open-source library for similarity search and clustering of dense vectors developed at Facebook AI. At run-time, we simply compute $v_q = E_Q(q)$ and retrieve the top $k$ passages with embeddings closest to $v_q$.

In this case, training the network means solving a **metric learning** problem: the two BERT networks need to learn an **effective vector space** such that relevant pairs of questions and passages are close, while irrelevant pairs are placed further away. In this problem we usually build a **training instance $D$** as ${(q_i, p_i^+, p_{i,1}^-, p_{i,2}^-, \dots, p_{i,n}^-)}^m_{i=1}$, where question $q$ is paired with a relevant (positive) passage $p_i^+$ and $n$ irrelevant (negative) passages. Then, the loss function is the negative log-likelihood of the positive passage:

$L(q_i, p_i^+, p_{i,1}^-, p_{i,2}^-, \dots, p_{i,n}^-) = -\log\frac{e^{sim(q_i, p_i^+)}}{e^{sim(q_i, p_i^+)} + \sum_{j=1}^n e^{sim(q_i, p_{i,j}^-)}}$

It's easy to find the positive paragraph, but choosing the negatives is quite important. In particular, the paper proposes different ways for sampling the negatives:
- Random: a negative is any random passage in the corpus
- TF-IDF (The paper uses a variant, BM25): the negatives are the top passages (not containing the answer) returned by a TF-IDF search
- Gold: the negatives are positives for other questions in the mini-batch. For the researchers, this is the best negative-mining option, because it's the most efficient and also it makes a batch a complete unit of learning (we learn the relationship that each question in the batch has with the other paragraphs).

The Gold method allows the **in-batch negatives** technique: assuming to have a batch size of $B$, then we collect two $B \times d$ matrices (one for questions, one for their positive paragraphs). Then, we compute $S = QP^\intercal$ which is a $B \times B$ matrix of **similarity scored** between each question and paragraph. This matrix can directly be used for training: any ($q_i, p_j$) pais where $i = j$ is considered to be a positive example, while it's negative otherwise. In total there will be $B$ training instances per batch, each with $B-1$ negative passages. 

# Configuration

## Imports

In [ ]:
import os
import numpy as np
import random
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, DistilBertTokenizer, \
                         TFBertModel, TFDistilBertModel
import utils

RANDOM_SEED = 42
BERT_DIMENSIONALITY = 768

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

### TPU check
The training could be made faster if we use the cloud GPUs offered by Google on Google Colab. Since TPUs require manual intialization and other oddities, we check multiple times throughout the notebook what kind of hardware we are running the code on.
using_TPU = True


In [ ]:
if using_TPU:
    try: 
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
        tf.config.experimental_connect_to_cluster(resolver)
        # This is the TPU initialization code that has to be at the beginning.
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print("All devices: ", tf.config.list_logical_devices('TPU'))
        strategy = tf.distribute.TPUStrategy(resolver)
    except:
        print("TPUs are not available, setting flag 'using_TPU' to False.")
        using_TPU = False
else:
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')
    checkpoint_dir = '/content/drive/My Drive/Uni/Magistrale/NLP/Project/weights/training_dpr/'
    datasets_dir = '/content/drive/My Drive/Uni/Magistrale/NLP/Project/datasets/dpr/'
else:
    # Create the folder where we'll save the weights of the model
    checkpoint_dir = os.path.join(ROOT_PATH, "data", "training_dpr")
    datasets_dir = os.path.join(ROOT_PATH, "data", "training_dpr", "dataset")

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(datasets_dir, exist_ok=True)

## Variables

We define all the paths.

In [ ]:
ROOT_PATH = os.path.dirname(os.getcwd())
TRAINING_FILE = os.path.join(ROOT_PATH, 'data', 'training_set.json')
VALIDATION_FILE = os.path.join(ROOT_PATH, 'data', 'validation_set.json')

We collect the training and validation questions and paragraphs into different lists.

In [ ]:
train_dict = utils.read_question_set(TRAINING_FILE)
val_dict = utils.read_question_set(VALIDATION_FILE)

def get_questions_and_paragraphs_from_dataset(dataset):
    questions = [{
            'qas': qas,
            'context_id': (i,j)    # We also track the question's original context and paragraph indices so to have a ground truth
        }
        for i in range(len(dataset['data']))
        for j, para in enumerate(dataset['data'][i]['paragraphs'])
        for qas in para['qas']
    ]

    paragraphs = [{
            'context': para['context'],
            'context_id': i
        }
        for i in range(len(dataset['data']))
        for para in dataset['data'][i]['paragraphs']
    ]

    return questions, paragraphs

train_questions, train_paragraphs = get_questions_and_paragraphs_from_dataset(train_dict)
val_questions, val_paragraphs = get_questions_and_paragraphs_from_dataset(val_dict)

We create the two different DistilBert models for encoding and test them on a random question/paragraph.

In [ ]:
tokenizer_distilbert = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_q, model_p = TFDistilBertModel.from_pretrained('distilbert-base-uncased'), \
                   TFDistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
test_question = train_questions[0]['qas']['question']
print(f"Testing on a simple question. \nQuestion: {test_question}")
inputs_test = tokenizer_distilbert(test_question, return_tensors="tf")
outputs = model_q(inputs_test)

# As a representation of the token we use the last hidden state at the [CLS] token (the first one)
last_hidden_states = outputs.last_hidden_state
test_q_repr = last_hidden_states[0,0,:]
print(f"Representation dimensionality: {test_q_repr.shape}")

# Training

First of all, we need to train our models. To do that, we need to create a dataset that feeds batches of questions and positive and negative paragraphs to a model, which is used to compute the representations, then the similarities and to correct the learnt distributions from the encoder models.

## Dataset creation

For the dataset, we use the `keras.utils.Sequence` object, which is a high-level multi-processing-ready data generator that we can use to generate full batches consisting of the tokenized question, as well as positive and negative paragraphs.

In [ ]:
import math
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from typing import List, Union
from tqdm import tqdm
from functools import partial

BATCH_SIZE = 8 if not using_TPU else 64

def get_paragraph_from_question(qas, dataset):
    i,j = qas['context_id']
    return dataset['data'][i]['paragraphs'][j]

def pre_tokenize_data(questions, dataset, tokenizer):
    tokenized_questions = [
        dict(tokenizer(questions[i]['qas']['question'], 
            max_length = 512, truncation = True, padding = 'max_length'))
    for i in tqdm(range(len(questions)))]
    tokenized_paragraphs = [
        dict(tokenizer(get_paragraph_from_question(
                    questions[i], dataset
                )['context'], max_length = 512, 
            truncation = True, padding = 'max_length'))
    for i in tqdm(range(len(questions)))]
    return tokenized_questions, tokenized_paragraphs

## Define the expected signature of the dataset:
features = {
    'input_ids': tf.TensorSpec(shape=(512,), dtype=tf.int32), 
    # 'token_type_ids': tf.TensorSpec(shape=(512,), dtype=tf.int32), # (BERT ONLY)
    'attention_mask': tf.TensorSpec(shape=(512,), dtype=tf.int32)
}

signature = {
    'questions': dict(features),
    'paragraphs': dict(features)
}

def dataset_generator(tokenized_questions, 
                      tokenized_paragraphs, 
                      dataset, tokenizer):
    # Generator for the dataset
    for i in range(len(tokenized_questions)):
        yield {
            'questions': tokenized_questions[i],
            'paragraphs': tokenized_paragraphs[i]
        }

def create_dataset_from_generator(questions, dataset, tokenizer):
    # Creates the dataset by means of the generator above.
    print("Pre-tokenizing data...")
    tok_questions, tok_paragraphs = pre_tokenize_data(questions, dataset, tokenizer)
    assert len(tok_questions) == len(tok_paragraphs), "Error while pre-tokenizing dataset"
    print("Generating dataset.")
    data_gen = partial(dataset_generator, tok_questions, tok_paragraphs, dataset, tokenizer)
    dataset = tf.data.Dataset.from_generator(data_gen, output_signature=dict(signature))
    dataset = dataset.cache()
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.apply(tf.data.experimental.assert_cardinality(len(questions)))
    return dataset

def decode_fn(record_bytes):
  example = tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {"question__input_ids": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "question__attention_mask": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__input_ids": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64),
       "paragraph__attention_mask": tf.io.FixedLenFeature(shape=(512,), dtype=tf.int64)})
  return {
      "questions": {'input_ids': example['question__input_ids'],
                   'attention_mask': example['question__attention_mask']},
      "paragraphs": {'input_ids': example['paragraph__input_ids'],
                   'attention_mask': example['paragraph__attention_mask']}
  }

def create_dataset_from_records(questions, dataset, tokenizer, fn, 
                                OVERRIDE=False, training=True):
    # Pre-tokenize and write dataset on disk
    filename = f'{fn}.proto'
    fn_type = filename.split(os.sep)[-1].replace('.proto','')
    gcs_filename = os.path.join('gs://volpepe-nlp-project-squad-datasets', f'{fn_type}.proto')
    if not os.path.exists(filename) or OVERRIDE:
        print("Pre-tokenizing data...")
        tok_questions, tok_paragraphs = pre_tokenize_data(questions, dataset, tokenizer)
        assert len(tok_questions) == len(tok_paragraphs), "Error while pre-tokenizing dataset"
        print("Saving dataset on disk...")
        with tf.io.TFRecordWriter(filename) as file_writer:
            for i in range(len(tok_questions)):
                record_bytes = tf.train.Example(features=tf.train.Features(feature={
                    "question__input_ids": tf.train.Feature(int64_list=tf.train.Int64List(
                            value=tok_questions[i]["input_ids"])),
                    "question__attention_mask": tf.train.Feature(int64_list=tf.train.Int64List(
                            value=tok_questions[i]["attention_mask"])),
                    "paragraph__input_ids": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=tok_paragraphs[i]["input_ids"])),
                    "paragraph__attention_mask": tf.train.Feature(int64_list=tf.train.Int64List(
                        value=tok_paragraphs[i]["attention_mask"]))
                    })).SerializeToString()
                file_writer.write(record_bytes)
        print("Upload the dataset on Google Cloud and re-run the function")
        return None
    print(f"Loading {fn_type} dataset from GCS ({gcs_filename}).")
    # Return it as processed dataset
    dataset = tf.data.TFRecordDataset([gcs_filename]).map(decode_fn)
    dataset = dataset.cache()
    if training:
        dataset = dataset.shuffle(10000)
        dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.apply(tf.data.experimental.assert_cardinality(len(questions)))
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

if not using_TPU:
    dataset_train = create_dataset_from_generator(train_questions, train_dict, tokenizer_distilbert)
    dataset_val = create_dataset_from_generator(val_questions, val_dict, tokenizer_distilbert)
else:
    dataset_train = create_dataset_from_records(train_questions, train_dict, tokenizer_distilbert, 
                                                os.path.join(datasets_dir, 'train'))
    dataset_val = create_dataset_from_records(val_questions, val_dict, tokenizer_distilbert,
                                                os.path.join(datasets_dir, 'val'), training=False)

## Training pipeline

First of all, we need a layer that takes as input the dictionary containing the tokenized questions and answers and returns their compact representations.

In [ ]:
class DenseEncoder(layers.Layer):
    def __init__(self, model_q, model_p):
        super().__init__()
        self.model_q = model_q  # Dense encoder for questions
        self.model_p = model_p  # Dense encoder for paragraphs
    
    def call(self, inputs, training=False):
        qs = inputs['questions']
        q_repr = self.model_q(qs).last_hidden_state[:,0,:]
        if training:
            # Input contains the questions and paragraphs encoding
            ps = inputs['paragraphs']
            p_repr = self.model_p(ps).last_hidden_state[:,0,:]
            return q_repr, p_repr
        else:
            return q_repr

# Small test for the layer
class TestDenseEncoderModel(keras.Model):
    def __init__(self, model_q, model_p):
        super().__init__()
        self.enc = DenseEncoder(model_q, model_p)

    def call(self, inputs, training=False):
        return self.enc(inputs, training=training)

test_model = TestDenseEncoderModel(model_q, model_p)
q_repr, p_repr = test_model(next(dataset_train.take(1).as_numpy_iterator()), training=True)
print(f"Output shape when in training mode: {q_repr.shape}, {p_repr.shape}")
q_repr_2 = test_model(next(dataset_train.take(1).as_numpy_iterator()), training=False)
print(f"Output shape when in testing mode: {q_repr_2.shape}")
q = tokenizer_distilbert(
    train_questions[0]['qas']['question'], max_length = 512, 
    truncation = True, padding = 'max_length', return_tensors="tf")
q_repr_3 = test_model({'questions': q})
print(f"Output shape when dealing with a single question: {q_repr_3.shape}")

Once we have the representations, we should compute the similarities, thus obtaining a a full mini-batch of positive-negative examples. 

In [ ]:
# Create the similarity matrix
S = tf.tensordot(q_repr, tf.transpose(p_repr), axes=1)
S.shape

This similarity matrix has the following meaning:
- Rows represent questions.
- Each row contains the similarity that the respective question has with the 16 paragraphs (one of them is the positive one, the others are negative)

In the paper, they refer to the loss as a *minimization of the negative log-likelihood of the positive passage*: what it really means is that we need to transform similarities to probabilities and use a categorical cross-entropy loss, where labels are the row index (which is also the column index in that row for the positive passage)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True
)
loss(y_true=tf.range(BATCH_SIZE), y_pred=S).numpy()

The loss seems to be quite high for this batch. We can study it with a confusion matrix.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

if not using_TPU: # Otherwise the batch size is HUGE
    S_arr = S.numpy()
    ConfusionMatrixDisplay.from_predictions(
        y_true=np.arange(BATCH_SIZE), y_pred=np.argmax(S_arr, axis=1))

Indeed, ideally the predictions should be on the diagonal. This means that the "default" space for this metric learning problem is not that good. We are ready to learn a new representation distribution.

## Model definition

In [ ]:
class DensePassageRetriever(keras.Model):

    def __init__(self, model_q, model_p):
        super().__init__()
        self.enc = DenseEncoder(model_q, model_p)

    def call(self, inputs, training=False):
        if training:
            # For training we return the similarity matrix
            repr_q, repr_p = self.enc(inputs, training=training)
            S = tf.tensordot(repr_q, tf.transpose(repr_p), axes=1)
            return S
        else:
            # In other cases, we return the representation of the question(s)
            repr_q = self.enc(inputs, training=training)            
            return repr_q

    def train_step(self, data):
        x = data
        y = tf.range(tf.shape(x['questions']['input_ids'])[0])
        with tf.GradientTape() as tape:
            # Obtain similarities
            S = self(x, training=True)
            # Obtain loss value
            loss = self.compiled_loss(y, S)
        # Construct gradients and apply them through the optimizer
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update and return metrics (specifically the one for the loss value).
        self.compiled_metrics.update_state(y, S)
        return {m.name: m.result() for m in self.metrics}

def create_model(sample, freeze_layers_up_to=5):
    print("Creating BERT models...")
    model_q, model_p =  TFDistilBertModel.from_pretrained('distilbert-base-uncased'), \
                        TFDistilBertModel.from_pretrained('distilbert-base-uncased')

    # Freeze layers 
    for i in range(freeze_layers_up_to): # layers 0 to variable are frozen, successive layers learn
        model_q.distilbert.transformer.layer[i].trainable = False
        model_p.distilbert.transformer.layer[i].trainable = False
    
    print("Creating Dense Passage Retriever...")
    model = DensePassageRetriever(model_q, model_p)

    print("Compiling...")
    # Compile the model and loss
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=3e-6),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )

    print("Testing on some data...")
    # Pass one batch of data to build the model
    model(sample)

    # Return the model
    print("Model created!")
    return model

## Training procedure

Define utility variables and saving paths.

In [ ]:
EPOCHS = 100
PATIENCE = 3

Check if we're using a TPU, in order to create the model within the scope of the strategy.

In [ ]:
import datetime

if using_TPU:
    # TPU requires to create the model within the scope of the distributed strategy
    # we're using.
    with strategy.scope():
        model = create_model(sample=next(dataset_train.take(1).as_numpy_iterator()),
                             freeze_layers_up_to=3)

    # Workaraound for saving locally when using cloud TPUs
    local_device_option = tf.train.CheckpointOptions(
        experimental_io_device="/job:localhost")
else:
    # GPUs and local systems don't need the above specifications. We simply
    # create a pattern for the filename and let the callbacks deal with it.
    checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:04d}.ckpt")
    # Also, on TPU we cannot use tensorboard, but on GPU we can
    log_dir = os.path.join(ROOT_PATH, "data", "logs", 
        "training_dpr", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    model = create_model(sample=next(dataset_train.take(1).as_numpy_iterator()))

    # ModelCheckpoint callback is only available when not using TPU
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        verbose=1,
        save_weights_only = True,
        save_best_only = False
    )

    # Same for tensorboard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1
    )

# Early stopping can be used by both hardware
es_callback = tf.keras.callbacks.EarlyStopping(
    patience = PATIENCE,
    restore_best_weights=True
)

if using_TPU:
    # Save first weights in a h5 file (it's the most stable way)
    model.save_weights(os.path.join(
        checkpoint_dir, 'training_normal_tpu_0.h5'),  overwrite=True)
else:
    # Save the first weights using the pattern from before
    model.save_weights(checkpoint_path.format(epoch=0))

callbacks = [es_callback]
if not using_TPU:
    # These callback imply saving stuff on local disk, which cannot be 
    # done automatically using TPUs.
    # Therefore, they are only active when using GPUs and local systems
    callbacks.extend([cp_callback, tensorboard_callback])

# We fit the model
history = model.fit(
    dataset_train, 
    y=None,
    validation_data=dataset_val,
    epochs=EPOCHS, 
    callbacks=callbacks,
    shuffle=True,
    use_multiprocessing=True,
    initial_epoch=0,
    verbose=1 # Show progress bar
)

if using_TPU:
    # Save last weights
    model.save_weights(os.path.join(
        checkpoint_dir, 'training_normal_tpu_last.h5'), overwrite=True)

## Testing

We pre-compute the representations of the paragraphs. We save it on disk so that we don't need to re-compute it everytime.

In [ ]:
# paragraphs_representations = np.empty(
#     shape=(len(paragraphs), BERT_DIMENSIONALITY)
# )
# paragraphs_representations.shape

In [ ]:
# for i, p in enumerate(tqdm(paragraphs)):
#     paragraphs_representations[i] = model_p(tokenizer_bert(
#         p['context'], return_tensors='tf', max_length = 512, 
#         truncation = True, padding = 'max_length'
#     )).last_hidden_state[0,0,:]
# np.savetxt('paragraphs_representations.txt', paragraphs_representations, delimiter=',')

In [ ]:
# paragraphs_representations = np.loadtxt('paragraphs_representations.txt', delimiter=',')